In [1]:
import json

import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

print plt.style.available
plt.style.use('ggplot')

import gensim.models

[u'seaborn-darkgrid', u'seaborn-notebook', u'classic', u'seaborn-ticks', u'grayscale', u'bmh', u'seaborn-talk', u'dark_background', u'ggplot', u'fivethirtyeight', u'seaborn-colorblind', u'seaborn-deep', u'seaborn-whitegrid', u'seaborn-bright', u'seaborn-poster', u'seaborn-muted', u'seaborn-paper', u'seaborn-white', u'seaborn-pastel', u'seaborn-dark', u'seaborn-dark-palette']


In [2]:
# --------------------------------------------- #
def fnShowNumberOfReviews(MOVIES):
    s = 0
    for m in MOVIES:
        if 'reviews' in m:
            # print m['name'], len(m['reviews'])
            s += len(m['reviews'])
        else:
            continue

    return s
    # --------------------------------------------- #

In [3]:
MOVIES = json.load(open('/Users/luuk/DATA/NUPINION---IMDB/data-blog---MASTER-v1313.json','r'))

print 'We have {} movies in the dataset.'.format(len(MOVIES))
print '\nNumber of reviews available: {}'.format(fnShowNumberOfReviews(MOVIES))

We have 9869 movies in the dataset.

Number of reviews available: 648988


In [4]:
print MOVIES[0].keys()

[u'poster_desc', u'reviews', u'description', u'avg_rating', u'year', u'poster_img', u'director', u'link', u'stars', u'num_votes', u'genre', u'runtime', u'num_reviews_pages', u'name']


In [5]:
REVIEWS = []

for movie in MOVIES:
    if 'reviews' in movie:
        reviews_tmp = []
        for r in movie['reviews']:
            r['movie'] = movie['name']
            r['link'] = movie['link']
            reviews_tmp.append(r)
        REVIEWS += reviews_tmp
    else:
        continue
        
print len(REVIEWS)
print json.dumps(REVIEWS[3],indent=4)

648988
{
    "movie": "Pulp Fiction", 
    "review_content": "\nPulp Fiction may be the single best film ever made, and quite\nappropriately it is by one of the most creative directors of all time,\nQuentin Tarantino. This movie is amazing from the beginning definition\nof pulp to the end credits and boasts one of the best casts ever\nassembled with the likes of Bruce Willis, Samuel L. Jackson, John\nTravolta, Uma Thurman, Harvey Keitel, Tim Roth and Christopher Walken.\nThe dialog is surprisingly humorous for this type of film, and I think\nthat's what has made it so successful. Wrongfully denied the many\nOscars it was nominated for, Pulp Fiction is by far the best film of\nthe 90s and no Tarantino film has surpassed the quality of this movie\n(although Kill Bill came close). As far as I'm concerned this is the\ntop film of all-time and definitely deserves a watch if you haven't\nseen it.\n", 
    "review_rating": 10, 
    "review_title": "Unbelievable.", 
    "link": "/title/tt01109

In [6]:
data = pd.DataFrame(REVIEWS)

print data.head(2)

                link         movie  \
0  /title/tt0110912/  Pulp Fiction   
1  /title/tt0110912/  Pulp Fiction   

                                      review_content  review_rating  \
0  \nOne of the early scenes in "Pulp Fiction" fe...             10   
1  \nTo put this in context, I am 34 years old an...             10   

                        review_title         user_href  \
0  The masterpiece without a message  /user/ur0556667/   
1                    Simply The Best  /user/ur1515595/   

                   user_name  
0  kylopod (kylopod@aol.com)  
1                  wolvesrug  


In [7]:
# Lets not hold massive objects in memory.
MOVIES = REVIEWS = None

---
### Select Sub-set

Since the dataset is to big...

In [8]:
print data.review_rating.value_counts()
print '\n', np.sum(data.review_rating.isnull())

data.review_rating.fillna(0, inplace=True)

10    130842
8      78948
9      71730
7      64099
1      53121
6      45018
5      34686
4      26655
3      25860
2      21939
Name: review_rating, dtype: int64

96090


In [9]:
# http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.StratifiedShuffleSplit.html
from sklearn.cross_validation import StratifiedShuffleSplit

ixs = StratifiedShuffleSplit(data.review_rating.values, n_iter=1, test_size=0.7, train_size=None, random_state=None)

print len(ixs)
train_ix, test_ix = [(i[0],i[1]) for i in ixs ][0]
print len(train_ix), len(test_ix)
ixs = None

1
194696 454292


---
### Multiword Expressions

https://radimrehurek.com/gensim/models/phrases.html#module-gensim.models.phrases

In [ ]:
bigram = gensim.models.Phrases(sentence_stream)

In [ ]:
>>> sent = [u'the', u'mayor', u'of', u'new', u'york', u'was', u'there']
>>> print(bigram[sent])
[u'the', u'mayor', u'of', u'new_york', u'was', u'there']

In [ ]:
>>> trigram = gensim.models.Phrases(bigram[sentence_stream])
>>> sent = [u'the', u'new', u'york', u'times', u'is', u'a', u'newspaper']
>>> print(trigram[bigram[sent]])
[u'the', u'new_york_times', u'is', u'a', u'newspaper']

---
### Word2Vec

https://radimrehurek.com/gensim/models/word2vec.html

In [ ]:
model = gensim.models.Word2Vec(size=100, window=5, min_count=5)

model.build_vocab(data.ix[train_ix].review_content.values)

%time model.train(data.ix[train_ix].review_content.values)

# To free memory...
model.init_sims(replace=True)

In [ ]:
model.n_similarity(["king"], ["duke"])

In [ ]:
# model.n_similarity(["king"], ["queen"])

# # text8-rest:
# > model.accuracy("questions-words.txt")

In [ ]:
# >>> model.most_similar(positive=['woman', 'king'], negative=['man'])
# [('queen', 0.50882536), ...]

# >>> model.doesnt_match("breakfast cereal dinner lunch".split())
# 'cereal'

# >>> model.similarity('woman', 'man')
# 0.73723527

# >>> model['computer']  # raw numpy vector of a word
# array([-0.00449447, -0.00310097,  0.02421786, ...], dtype=float32)